In [283]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.modules.distance import PairwiseDistance

In [284]:
epl = WordEmbeddings('pl')

In [285]:
output = pd.read_csv("output.csv")

In [286]:
data = pd.read_csv("input.csv")

In [287]:
output.columns = ['o1', 'o2']

In [288]:
data = pd.concat([data, output], axis=1)

In [289]:
data.drop_duplicates().isna().sum()

verb                              0
verb - main semantic class        0
verb - second semantic class    403
verb - third semantic class     576
verb - tense                      0
o1                                0
o2                                0
dtype: int64

In [290]:
data1 = data.drop_duplicates()

In [291]:
data1.columns = ['verb', 'c1', 'c2', 'c3', 'tense', 'o1', 'o2']

In [292]:
full = data1[~data1.c3.isna()]
noc3 = data1[data1.c3.isna()]

In [293]:
noc3c2 = noc3[~data1.c2.isna()]
noc3noc2 = noc3[data1.c2.isna()]

C:\Users\Piotr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\Piotr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [294]:
noc3c2

,verb,c1,c2,c3,tense,o1,o2
3,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,past,"""+""","""+"""
4,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,present,o,o
6,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,past,o,o
20,"[p] poczuć, że_",percepcyjny,epistemiczny,NaN,past,"""+""","""+"""
24,"bać się, że_",emotywny,epistemiczny,NaN,present,o,o
...,...,...,...,...,...,...,...
2556,"[e] widzieć, że_",epistemiczny,percepcyjny,NaN,present,"""+""","""+"""
2560,"[e] widzieć, że_",epistemiczny,percepcyjny,NaN,brak,"""+""","""+"""
2579,"[p] widać, że_",percepcyjny,epistemiczny,NaN,brak,"""+""","""+"""
2581,"[p] widać, że_",percepcyjny,epistemiczny,NaN,past,"""+""","""+"""


In [295]:
noc3noc2.c2 = noc3noc2.c1
noc3noc2.c3 = noc3noc2.c1

C:\Users\Piotr\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [296]:
noc3c2

,verb,c1,c2,c3,tense,o1,o2
3,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,past,"""+""","""+"""
4,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,present,o,o
6,"[e] czuć, że_",epistemiczny,percepcyjny,NaN,past,o,o
20,"[p] poczuć, że_",percepcyjny,epistemiczny,NaN,past,"""+""","""+"""
24,"bać się, że_",emotywny,epistemiczny,NaN,present,o,o
...,...,...,...,...,...,...,...
2556,"[e] widzieć, że_",epistemiczny,percepcyjny,NaN,present,"""+""","""+"""
2560,"[e] widzieć, że_",epistemiczny,percepcyjny,NaN,brak,"""+""","""+"""
2579,"[p] widać, że_",percepcyjny,epistemiczny,NaN,brak,"""+""","""+"""
2581,"[p] widać, że_",percepcyjny,epistemiczny,NaN,past,"""+""","""+"""


In [297]:
noc3c2.c3 = noc3c2.c2
noc3c2.c2 = noc3c2.c1

C:\Users\Piotr\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [298]:
full

,verb,c1,c2,c3,tense,o1,o2
214,"dowieść, że_",dowodzenia,wnioskowania,mówienia,past,"""+""",o
219,"dowieść, że_",dowodzenia,wnioskowania,mówienia,past,"""+?""",o
220,"dowodzić, że_",dowodzenia,wnioskowania,mówienia,present,o,o
221,"dowodzić, że_",dowodzenia,wnioskowania,mówienia,past,o,o
741,"obawiać się, że_",emotywny,epistemiczny,wolicjonalny,past,o,o
742,"obawiać się, że_",emotywny,epistemiczny,wolicjonalny,present,o,o
750,"obawiać się, że_",emotywny,epistemiczny,wolicjonalny,future,o,o
775,"oceniać, że_",epistemiczny,wnioskowania,epistemiczny,present,o,o
778,"ocenić, że_",mówienia,wnioskowania,epistemiczny,past,o,o
1034,"pokazać, że_",mówienia,wnioskowania,epistemiczny,past,o,o


In [299]:
noc3c2

,verb,c1,c2,c3,tense,o1,o2
3,"[e] czuć, że_",epistemiczny,epistemiczny,percepcyjny,past,"""+""","""+"""
4,"[e] czuć, że_",epistemiczny,epistemiczny,percepcyjny,present,o,o
6,"[e] czuć, że_",epistemiczny,epistemiczny,percepcyjny,past,o,o
20,"[p] poczuć, że_",percepcyjny,percepcyjny,epistemiczny,past,"""+""","""+"""
24,"bać się, że_",emotywny,emotywny,epistemiczny,present,o,o
...,...,...,...,...,...,...,...
2556,"[e] widzieć, że_",epistemiczny,epistemiczny,percepcyjny,present,"""+""","""+"""
2560,"[e] widzieć, że_",epistemiczny,epistemiczny,percepcyjny,brak,"""+""","""+"""
2579,"[p] widać, że_",percepcyjny,percepcyjny,epistemiczny,brak,"""+""","""+"""
2581,"[p] widać, że_",percepcyjny,percepcyjny,epistemiczny,past,"""+""","""+"""


In [300]:
noc3noc2

,verb,c1,c2,c3,tense,o1,o2
0,[elipsa],mówienia,mówienia,mówienia,brak,o?,?
1,[elipsa],epistemiczny,epistemiczny,epistemiczny,past,?,?
2,[elipsa],mówienia,mówienia,mówienia,past,?,?
21,"alarmować, że_",mówienia,mówienia,mówienia,past,o,o
22,"argumentować, że_",mówienia,mówienia,mówienia,past,o,o
...,...,...,...,...,...,...,...
2541,"[e] widać, że",epistemiczny,epistemiczny,epistemiczny,present,"""+""","""+"""
2542,"[e] widać, że",epistemiczny,epistemiczny,epistemiczny,brak,"""+""","""+"""
2551,"[p] zauważyć, że_",percepcyjny,percepcyjny,percepcyjny,past,"""+""","""+"""
2561,"[e] widzieć, że_",epistemiczny,epistemiczny,epistemiczny,past,"""+""","""+"""


In [301]:
final = full.append(noc3c2).append(noc3noc2)

In [302]:
final

,verb,c1,c2,c3,tense,o1,o2
214,"dowieść, że_",dowodzenia,wnioskowania,mówienia,past,"""+""",o
219,"dowieść, że_",dowodzenia,wnioskowania,mówienia,past,"""+?""",o
220,"dowodzić, że_",dowodzenia,wnioskowania,mówienia,present,o,o
221,"dowodzić, że_",dowodzenia,wnioskowania,mówienia,past,o,o
741,"obawiać się, że_",emotywny,epistemiczny,wolicjonalny,past,o,o
...,...,...,...,...,...,...,...
2541,"[e] widać, że",epistemiczny,epistemiczny,epistemiczny,present,"""+""","""+"""
2542,"[e] widać, że",epistemiczny,epistemiczny,epistemiczny,brak,"""+""","""+"""
2551,"[p] zauważyć, że_",percepcyjny,percepcyjny,percepcyjny,past,"""+""","""+"""
2561,"[e] widzieć, że_",epistemiczny,epistemiczny,epistemiczny,past,"""+""","""+"""


In [303]:
final.o1.value_counts()

o       378
"+"     201
"-"      15
"+?"     12
o?        8
?         2
Name: o1, dtype: int64

In [304]:
final.o1[final.o1.values == '"+"']  = '+'

In [305]:
final.o1[final.o1.values == '"-"']  = '-'

In [306]:
final.o1[final.o1.values == '"+?"'] = '+'

In [307]:
final.o1[final.o1.values == 'o?']   = 'o'

In [308]:
final = final[final.o1 != '?']

In [309]:
final.o2[final.o2.values == '"+"']  = '+'

C:\Users\Piotr\Anaconda3\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [310]:
final.o2[final.o2.values == '"-"']  = '-'

In [311]:
final.o2[final.o2.values == '"+?"'] = '+'

In [312]:
final.o2[final.o2.values == 'o?']   = 'o'

In [313]:
final = final[final.o2 != '?']

In [314]:
final = final[final.o2 != '1?']

In [315]:
final.o2.value_counts()

o    394
+    171
-      7
Name: o2, dtype: int64

In [316]:
final = final[final.c3 != "performatyw"]

In [347]:
d1 = pd.get_dummies(final.o1)
d2 = pd.get_dummies(final.o2)
dc1 = pd.get_dummies(final.c1)
dc2 = pd.get_dummies(final.c2)
dc3 = pd.get_dummies(final.c3)

d1.columns = ['o1+', 'o1-', 'o1o']
d2.columns = ['o2+', 'o2-', 'o2o']

In [348]:
dc1.columns

Index(['?', 'czynnościowy', 'dowodzenia', 'emotywny', 'epistemiczny',
       'liczenia', 'mówienia', 'odkrycia', 'pamięciowy', 'percepcyjny',
       'przyczynowy', 'wnioskowania', 'zdarzeniowy'],
      dtype='object')

In [349]:
dc2.columns

Index(['?', 'czynnościowy', 'dowodzenia', 'emotywny', 'epistemiczny',
       'liczenia', 'mówienia', 'odkrycia', 'pamięciowy', 'percepcyjny',
       'przyczynowy', 'wnioskowania', 'zdarzeniowy'],
      dtype='object')

In [350]:
dc3.columns

Index(['?', 'czynnościowy', 'emotywny', 'epistemiczny', 'mówienia',
       'nie-wiedzowy', 'odkrycia', 'pamięciowy', 'percepcyjny', 'przyczynowy',
       'wnioskowania', 'wolicjonalny', 'wolitywny', 'zdarzeniowy'],
      dtype='object')

In [351]:
dc3['dowodzenia'] = 0
dc3['epistemiczny'] = 0
dc3['liczenia'] = 0

In [352]:
dc2['nie-wiedzowy'] = 0
dc2['wolicjonalny'] = 0
dc2['wolitywny'] = 0

In [353]:
dc1['nie-wiedzowy'] = 0
dc1['wolicjonalny'] = 0
dc1['wolitywny'] = 0

In [354]:
dc1.columns

Index(['?', 'czynnościowy', 'dowodzenia', 'emotywny', 'epistemiczny',
       'liczenia', 'mówienia', 'odkrycia', 'pamięciowy', 'percepcyjny',
       'przyczynowy', 'wnioskowania', 'zdarzeniowy', 'nie-wiedzowy',
       'wolicjonalny', 'wolitywny'],
      dtype='object')

In [355]:
dc2.columns

Index(['?', 'czynnościowy', 'dowodzenia', 'emotywny', 'epistemiczny',
       'liczenia', 'mówienia', 'odkrycia', 'pamięciowy', 'percepcyjny',
       'przyczynowy', 'wnioskowania', 'zdarzeniowy', 'nie-wiedzowy',
       'wolicjonalny', 'wolitywny'],
      dtype='object')

In [356]:
dc3.columns

Index(['?', 'czynnościowy', 'emotywny', 'epistemiczny', 'mówienia',
       'nie-wiedzowy', 'odkrycia', 'pamięciowy', 'percepcyjny', 'przyczynowy',
       'wnioskowania', 'wolicjonalny', 'wolitywny', 'zdarzeniowy',
       'dowodzenia', 'liczenia'],
      dtype='object')

In [357]:
dc = dc1 + dc2 + dc3

In [359]:
finalfinal = pd.concat([final.verb, dc, d1, d2], axis=1)

In [370]:
finalfinal.columns = ['verb', '?', 'czynnościowy', 'dowodzenia', 'emotywny', 'epistemiczny',
       'liczenia', 'mówienia', 'nie_wiedzowy', 'odkrycia', 'pamięciowy',
       'percepcyjny', 'przyczynowy', 'wnioskowania', 'wolicjonalny',
       'wolitywny', 'zdarzeniowy', 'o1+', 'o1-', 'o1o', 'o2+', 'o2-', 'o2o']

In [372]:
finalfinal.to_csv("./finalDataset.csv")